In [11]:
import pandas as pd
import re
import os

### Transform xlsx file to txt

In [ ]:


def convert_xlsx_to_text(xlsx_file, output_file, semester="Fall 2025"):
    try:
        print(f"Reading Excel file: {xlsx_file}")
        
        # Import openpyxl for direct access to hyperlinks
        from openpyxl import load_workbook
        
        # Load the workbook directly to access hyperlinks
        wb = load_workbook(xlsx_file)
        ws = wb.active
        
        # Create a mapping for syllabus column and hyperlinks
        syllabus_col_idx = None
        header_row = 1  # Assuming the header is in row 1
        
        # Find the syllabus column index
        for col in range(1, ws.max_column + 1):
            cell_value = ws.cell(row=header_row, column=col).value
            if cell_value and "Syllabus" in str(cell_value):
                syllabus_col_idx = col
                break
        
        # Create a dictionary to store hyperlinks by row
        syllabus_links = {}
        
        # Find all hyperlinks in the syllabus column
        if syllabus_col_idx:
            for row in range(header_row + 1, ws.max_row + 1):
                cell = ws.cell(row=row, column=syllabus_col_idx)
                
                # Check if cell has a hyperlink
                if cell.hyperlink:
                    syllabus_links[row] = {
                        'text': cell.value,
                        'url': cell.hyperlink.target
                    }
        
        print(f"Found {len(syllabus_links)} hyperlinks in the syllabus column")
        
        # Now read with pandas for data processing
        df = pd.read_excel(xlsx_file)
        
        # Clean column names in case there are extra spaces or line breaks
        df.columns = [col.strip() if isinstance(col, str) else col for col in df.columns]
        
        # Group courses by department (Subject)
        departments = df["Subject"].unique()
        
        # Write text output
        with open(f"{output_file}.txt", "w", encoding="utf-8") as f:
            for department in departments:
                dept_courses = df[df["Subject"] == department]
                
                for idx, course in dept_courses.iterrows():
                    course_text = []
                    
                    # Basic course information
                    course_text.append(f"Semester: {semester}")
                    course_text.append(f"Department: {department}")
                    course_text.append(f"Course: {course['Subject']} {course['Catalog #']}")
                    
                    # Extract title (clean up newlines if present)
                    title_col = [col for col in df.columns if "Course Title Long" in col][0]
                    title = course[title_col]
                    if pd.notna(title) and "\r\n" in str(title):
                        title = str(title).split("\r\n")[0]  # Take just the long title
                    course_text.append(f"Title: {title}")
                    
                    # Credits/Units
                    course_text.append(f"Units: {course['Credits']}")
                    
                    # Previous Course Number
                    old_num_col = [col for col in df.columns if "Pre-Fall 2025" in col]
                    if old_num_col and pd.notna(course[old_num_col[0]]):
                        course_text.append(f"Previous Number: {course[old_num_col[0]]}")
                    
                    # Enforced Prerequisites
                    prereq_col = [col for col in df.columns if "Enforced Prerequisites" in col]
                    if prereq_col and pd.notna(course[prereq_col[0]]):
                        course_text.append(f"Enforced Prerequisites: {course[prereq_col[0]]}")
                    
                    # Advisory Prerequisites
                    adv_prereq_col = [col for col in df.columns if "Advisory Prerequisites" in col]
                    if adv_prereq_col and pd.notna(course[adv_prereq_col[0]]):
                        course_text.append(f"Advisory Prerequisites: {course[adv_prereq_col[0]]}")
                    
                    # Course Description
                    desc_col = [col for col in df.columns if "Description" in col]
                    if desc_col and pd.notna(course[desc_col[0]]):
                        course_text.append(f"Description: {course[desc_col[0]]}")
                    
                    # Typical Term Offered
                    typical_term_col = [col for col in df.columns if "Typical Term" in col]
                    if typical_term_col and pd.notna(course[typical_term_col[0]]):
                        course_text.append(f"Typically Offered: {course[typical_term_col[0]]}")
                    
                    # Last Term Offered
                    last_term_col = [col for col in df.columns if "Last Term" in col]
                    if last_term_col and pd.notna(course[last_term_col[0]]):
                        course_text.append(f"Last Offered: {course[last_term_col[0]]}")
                    
                    # Notes
                    notes_col = [col for col in df.columns if "Notes" in col]
                    if notes_col and pd.notna(course[notes_col[0]]):
                        course_text.append(f"Notes: {course[notes_col[0]]}")
                    
                    # Syllabus Link - using our extracted hyperlinks
                    syllabus_col = [col for col in df.columns if "Syllabus" in col]
                    if syllabus_col and pd.notna(course[syllabus_col[0]]):
                        syllabus_text = course[syllabus_col[0]]
                        
                        # Excel rows are 1-indexed and we need to account for the header
                        excel_row = idx + 2  # +2 because we have a 0-indexed DataFrame and Excel is 1-indexed
                        
                        # Check if we have a hyperlink for this row
                        if excel_row in syllabus_links:
                            link_info = syllabus_links[excel_row]
                            course_text.append(f"Syllabus: {syllabus_text} (Link: {link_info['url']})")
                        else:
                            course_text.append(f"Syllabus: {syllabus_text}")
                    
                    # Cross Listed
                    cross_col = [col for col in df.columns if "Cross Listed" in col]
                    if cross_col and pd.notna(course[cross_col[0]]):
                        course_text.append(f"Cross Listed: {course[cross_col[0]]}")
                    
                    # Home Department
                    home_col = [col for col in df.columns if "Home Department" in col]
                    if home_col and pd.notna(course[home_col[0]]):
                        course_text.append(f"Home Department: {course[home_col[0]]}")
                    
                    # Primary Requisite
                    primary_req_col = [col for col in df.columns if "PRIMARY" in col]
                    if primary_req_col and pd.notna(course[primary_req_col[0]]):
                        course_text.append(f"Primary Requisite: {course[primary_req_col[0]]}")
                    
                    # Additional Requisite
                    add_req_col = [col for col in df.columns if "Additional" in col and "Requisite" in col]
                    if add_req_col and pd.notna(course[add_req_col[0]]):
                        course_text.append(f"Additional Requisite: {course[add_req_col[0]]}")
                    
                    # Write this course block
                    f.write("\n".join(course_text) + "\n\n")
        
        print(f"Conversion complete. Output saved to {output_file}.txt")
    
    except Exception as e:
        print(f"Error processing Excel file: {e}")
        import traceback
        traceback.print_exc()



In [14]:
def main():
    """
    Main function to convert all xlsx files in courses/soc directory to txt files
    """
    # Input directory path
    input_directory = "/Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc"
    
    # Get all xlsx files in the directory
    xlsx_files = [f for f in os.listdir(input_directory) if f.endswith('.xlsx')]
    
    if not xlsx_files:
        print(f"No xlsx files found in {input_directory}.")
        return
    
    # Process each xlsx file
    for xlsx_file in xlsx_files:
        input_path = os.path.join(input_directory, xlsx_file)
        # Create output filename by replacing .xlsx with .txt
        output_filename = os.path.splitext(xlsx_file)[0] + '.txt'
        output_path = os.path.join(input_directory, output_filename)
        
        print(f"Converting {input_path} to {output_path}...")
        convert_xlsx_to_text(input_path, output_path)
    
    print("Conversion completed successfully!")


if __name__ == "__main__":
    main()

Converting /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/~$umsi_bsi_course.xlsx to /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/~$umsi_bsi_course.txt...
Reading Excel file: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/~$umsi_bsi_course.xlsx
Error processing Excel file: File is not a zip file
Converting /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/umsi_studyabroad_course.xlsx to /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/umsi_studyabroad_course.txt...
Reading Excel file: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/umsi_studyabroad_course.xlsx
Found 0 hyperlinks in the syllabus column
Conversion complete. Output saved to /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/umsi_studyabroad_course.txt.txt
Converting /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/umsi_bsi_course.xlsx to /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/umsi_bsi_course.txt...
Reading Excel file: /Users/yixin/Deskt

Traceback (most recent call last):
  File "/var/folders/mc/f8k2g59s045ggmnrknqbjv0h0000gn/T/ipykernel_62219/921650908.py", line 18, in convert_xlsx_to_text
    wb = load_workbook(xlsx_file)
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/reader/excel.py", line 346, in load_workbook
    reader = ExcelReader(filename, read_only, keep_vba,
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/reader/excel.py", line 123, in __init__
    self.archive = _validate_archive(fn)
                   ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/reader/excel.py", line 95, in _validate_archive
    archive = ZipFile(filename, 'r')
              ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/zipfile.py

### Transform CSV file to txt

In [15]:
import os
import csv
import re
import glob
from datetime import datetime

In [16]:

def convert_csv_to_text(csv_file, output_file=None, semester=None):
    """
    Convert a CSV file containing course data to a formatted text file.
    Handles the specific format with headers like Term, Session, Subject, etc.
    
    Parameters:
    csv_file (str): Path to the CSV file
    output_file (str): Path to save the output file (without extension)
    semester (str): Semester to use in the output, defaults to derived from the file
    """
    try:
        print(f"Processing CSV file: {csv_file}")
        
        # If no output file specified, create one based on input filename
        if output_file is None:
            output_file = os.path.splitext(csv_file)[0] + "_output"
        
        # Read the CSV file
        with open(csv_file, 'r', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
        
        # Extract semester from the file if not provided
        if semester is None:
            # Try to get semester from the first row's Term field
            if rows and 'Term' in rows[0]:
                term_value = rows[0]['Term']
                # Convert term code (like 2210) to a readable semester
                if term_value and re.match(r'^\d{4}$', term_value):
                    # Last digit is typically: 0=Winter, 4=Spring, 6=Summer, 8=Fall
                    year = '20' + term_value[:2]
                    term_digit = term_value[-1]
                    
                    if term_digit == '0':
                        semester = f"Winter {year}"
                    elif term_digit == '4':
                        semester = f"Spring {year}"
                    elif term_digit == '6':
                        semester = f"Summer {year}"
                    elif term_digit == '8':
                        semester = f"Fall {year}"
                    else:
                        semester = f"Term {term_value}"
                else:
                    semester = term_value
            
            # Fallback to current year if still not determined
            if not semester:
                current_year = datetime.now().year
                semester = f"Fall {current_year}"
        
        # Group courses by subject and catalog number to combine sections
        courses = {}
        
        for row in rows:
            # Create a key for each unique course
            subject = row.get('Subject', '')
            catalog = row.get('Catalog Nbr', '')
            
            if not subject or not catalog:
                continue
                
            course_key = f"{subject} {catalog}"
            
            # If this is a new course, initialize it
            if course_key not in courses:
                courses[course_key] = {
                    'department': subject,
                    'number': catalog,
                    'title': row.get('Course Title', ''),
                    'units': row.get('Units', ''),
                    'sections': []
                }
            
            # Add section information
            section = {
                'lec_sec': row.get('Section', ''),
                'component': row.get('Component', ''),
                'days': get_days_string(row),
                'begin': get_time_part(row.get('Time', ''), is_start=True),
                'end': get_time_part(row.get('Time', ''), is_start=False),
                'room': row.get('Location', ''),
                'location': row.get('Location', ''),  # May need parsing if location contains building and room
                'instructor': row.get('Instructor', ''),
                'start_date': row.get('Start Date', ''),
                'end_date': row.get('End Date', ''),
                'class_nbr': row.get('Class Nbr', ''),
                'session': row.get('Session', ''),
                'seats_total': row.get('Total', ''),
                'seats_remaining': row.get('Seats Remaining', ''),
                'waitlist': row.get('Has WL', '')
            }
            
            courses[course_key]['sections'].append(section)
        
        # Write text output
        with open(f"{output_file}.txt", "w", encoding="utf-8") as f:
            for course_key, course_data in courses.items():
                course_text = []
                
                # Basic course information
                course_text.append(f"Semester: {semester}")
                course_text.append(f"Department: {course_data['department']}")
                course_text.append(f"Course: {course_key}")
                course_text.append(f"Title: {course_data['title']}")
                course_text.append(f"Units: {course_data['units']}")
                course_text.append("Sections")
                
                # Section information
                for section in course_data['sections']:
                    course_text.append(f"Lec/Sec: {section['lec_sec']}")
                    course_text.append(f"Component: {section['component']}")
                    course_text.append(f"Class Number: {section['class_nbr']}")
                    course_text.append(f"Days: {section['days']}")
                    course_text.append(f"Begin: {section['begin']}")
                    course_text.append(f"End: {section['end']}")
                    course_text.append(f"Date Range: {section['start_date']} - {section['end_date']}")
                    course_text.append(f"Room: {section['room']}")
                    course_text.append(f"Location: {section['location']}")
                    course_text.append(f"Instructor: {section['instructor']}")
                    
                    if section['seats_total'] or section['seats_remaining']:
                        course_text.append(f"Enrollment: {section['seats_total']} total, {section['seats_remaining']} remaining")
                        
                    if section['waitlist']:
                        course_text.append(f"Waitlist Available: {section['waitlist']}")
                        
                    course_text.append("")
                
                # Write this course block
                f.write("\n".join(course_text) + "\n\n")
        
        print(f"Conversion complete. Output saved to {output_file}.txt")
        return output_file + ".txt"
    
    except Exception as e:
        print(f"Error processing CSV file: {e}")
        import traceback
        traceback.print_exc()
        return None




In [17]:
def get_days_string(row):
    """
    Extract the days from the M, T, W, TH, F, S, SU columns
    """
    days = []
    day_columns = ['M', 'T', 'W', 'TH', 'F', 'S', 'SU']
    
    for day in day_columns:
        if day in row and row[day] and row[day].strip():
            days.append(day)
    
    return " ".join(days)


def get_time_part(time_str, is_start=True):
    """
    Extract start or end time from a time range string like "10:00AM - 11:30AM"
    """
    if not time_str:
        return ""
        
    match = re.search(r'(\d+:\d+\s*[AP]M)\s*-\s*(\d+:\d+\s*[AP]M)', time_str)
    if match:
        return match.group(1) if is_start else match.group(2)
    return time_str

In [18]:
def process_all_csv_files(directory='.', output_dir=None):
    """
    Process all CSV files in the given directory
    """
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    csv_files = glob.glob(os.path.join(directory, '*.csv'))
    results = []
    
    for csv_file in csv_files:
        print(f"\nProcessing: {csv_file}")
        if output_dir:
            base_name = os.path.basename(csv_file)
            output_file = os.path.join(output_dir, os.path.splitext(base_name)[0])
        else:
            output_file = os.path.splitext(csv_file)[0] + "_output"
            
        result = convert_csv_to_text(csv_file, output_file)
        if result:
            results.append(result)
    
    return results



In [20]:

def main():
    """
    Main function to execute the script
    """
    # Specify the source directory for CSV files
    source_dir = '/Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc'
    
    # Specify output directory (can be None to output alongside source files)
    output_dir = '/Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc'
    
    # Ensure output directory exists
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Process all CSV files in the specified directory
    print(f"Processing all CSV files in {source_dir}")
    process_all_csv_files(source_dir, output_dir)


if __name__ == "__main__":
    main()

Processing all CSV files in /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc

Processing: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/SP2025_open.csv
Processing CSV file: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/SP2025_open.csv
Conversion complete. Output saved to /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/SP2025_open.txt

Processing: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/WN2025_open_all.csv
Processing CSV file: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/WN2025_open_all.csv
Conversion complete. Output saved to /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/WN2025_open_all.txt

Processing: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/FA2025_open.csv
Processing CSV file: /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/FA2025_open.csv
Conversion complete. Output saved to /Users/yixin/Desktop/SI 630/Project_RAG/data/courses/soc/FA2025_open.txt

Processing: /Users/yixin/Desktop